### Extract timing files

In [1]:
#r "nuget: System.Data.SQLite, *-*"
#r "nuget: Microsoft.DotNet.Interactive.ExtensionLab, *-*"
#r "nuget: Dapper, *-*"

using System.IO;
using System.Text.Json;
using System.Text.Encodings.Web;
using System.Text.Unicode;
using System.Data.SQLite;
using Dapper;

var jsonSerializerOptions = new JsonSerializerOptions
{
    Encoder = JavaScriptEncoder.Create(UnicodeRanges.All),
    //WriteIndented = true
};

#### Recitation Timing (Word by word)

In [3]:
var dbPath = @"D:\DatabaseBackups\QuranDB\recitaion-timings\";

record WbwRecitationTiming(long sura, long ayah, long time, string words);

bool SaveAsJson(string dbFilename){
    var dbFileFullName = dbPath + dbFilename + ".db";
    
    if (!File.Exists(dbFileFullName)) 
        return false;

    using (var connenction = new SQLiteConnection($"Data Source={dbFileFullName};Mode=Memory;Cache=Shared"))
    {
        var result = connenction.Query<WbwRecitationTiming>("SELECT sura, ayah, time, words FROM timings ORDER BY sura, ayah");

        var arrayList = new List<object>();

        foreach (var item in result){
            var words = item.words.Split(',');
            var wordTimes = words.Select(w=>w.Split(':').Select(s=>Convert.ToInt32(s)));
            arrayList.Add(new object[]{ item.sura, item.ayah, item.time, wordTimes, "newLine" });
        }

        var resultJson = JsonSerializer.Serialize(arrayList, jsonSerializerOptions)
                            .Replace(",\"newLine\"],", "]," + Environment.NewLine)
                            .Replace(",\"newLine\"","");
    
        File.WriteAllText(dbPath + "extractedJson\\" + dbFilename + ".json", resultJson);
    }

    return true;
}

SaveAsJson("mishari_alafasy");

#### Recitation Timing (Ayat by Ayat)

In [5]:
var dbPath = @"D:\DatabaseBackups\QuranDB\recitaion-timings\";

record RecitationTiming(Int32 sura, Int32 ayah, Int32 time);

bool SaveAsJson(string dbFilename){
    var dbFileFullName = dbPath + dbFilename + ".db";
    
    if (!File.Exists(dbFileFullName)) 
        return false;

    using (var connenction = new SQLiteConnection($"Data Source={dbFileFullName};Mode=Memory;Cache=Shared"))
    {
        var result = connenction.Query<RecitationTiming>("SELECT sura, ayah, time FROM timings ORDER BY sura, ayah");

        var arrayList = new List<object>();

        foreach (var item in result){       
            if (item.ayah != 999)     
                arrayList.Add(new object[]{ item.sura, item.ayah, item.time, "newLine" });
        }

        var resultJson = JsonSerializer.Serialize(arrayList, jsonSerializerOptions)
                            .Replace(",\"newLine\"],", "]," + Environment.NewLine)
                            .Replace(",\"newLine\"","");
    
        File.WriteAllText(dbPath + "extractedJson\\" + dbFilename + ".json", resultJson);
    }

    return true;
}

SaveAsJson("muaiqly_kfgqpc");

### Download timing files form qurancdn.com

Reciters (id, name)

- 1 abdul_baset/mujawwad
- 2 abdul_baset/murattal
- 3 abdurrahmaan_as_sudais/murattal
- 4 abu_bakr_shatri/murattal
- 5 hani_ar_rifai/murattal
- 6 khalil_al_husary/murattal
- 7 mishari_al_afasy/murattal
- 8 siddiq_al-minshawi/mujawwad
- 9 siddiq_minshawi/murattal
- 10 saud_ash-shuraym/murattal
- 11 abdul_muhsin_alqasim
- 12 khalil_al_husary/muallim
- 13 sa3d_al-ghaamidi/complete/
- 14 fares
- 17 sahl_yaaseen
- 18 salaah_bukhaatir/
- 19 ahmed_ibn_3ali_al-3ajamy/
- 43 salahbudair/
- 44 aziz_alili/
- 58 mishaari_w_ibrahim_walk_si/
- 66 abdullah_basfar_w_ibrahim_walk_si/
- 88 mostafa_ismaeel/
- 91 mohammad_altablawi/
- 95 mishaari_california/
- 97 yasser_ad-dussary/
- 104 nasser_bin_ali_alqatami/
- 122 mahmood_khaleel_al-husaree_iza3a/
- 124 abdullah_matroud/
- 126 ahmad_nauina/
- 127 akram_al_alaqmi/
- 128 ali_hajjaj_alsouasi/
- 129 mahmood_ali_albana/
- 158 ali_jaber/
- 159 maher_almu3aiqly/year1440/
- 160 bandar_baleela/complete/
- 161 khalifah_taniji/murattal
- 162 abdullaah_3awwaad_al-juhaynee/
- 163 abdullaah_basfar/
- 168 siddiq_minshawi/kids_repeat
- 169 muhammad_jibreel/complete/
- 170 khalid_jalil/murattal/mp3
- 172 hadi_toure/mp3
- 173 mishari_al_afasy/streaming/mp3
- 174 yasser_ad-dussary/mp3

In [7]:
#r "nuget: System.Net.Http.Json, *-*"

Installed Packages System.Net.Http.Json, 9.0.0-preview.5.24306.7

In [16]:
using System.IO;
using System.Text.Json;
using System.Net.Http;
using System.Net.Http.Json;
using System.Text.Encodings.Web;
using System.Text.Unicode;

var jsonSerializerOptions = new JsonSerializerOptions
{
    Encoder = JavaScriptEncoder.Create(UnicodeRanges.All),
    WriteIndented = true
};

var srcPath = @"D:\Faruque\Projects\quranpwa\src\assets\";
var quranDataJson = File.ReadAllText(srcPath + "quran-data.json");

public class QuranData
{
    public List<List<object>> suras { get; set; }
    public List<List<int>> hizb_quarters { get; set; }
    public List<List<int>> manzils { get; set; }
    public List<List<int>> rukus { get; set; }
    public List<List<int>> pages { get; set; }
    public List<List<object>> sajdas { get; set; }
    public List<List<int>> juzs { get; set; }
}

var quranData = JsonSerializer.Deserialize<QuranData>(quranDataJson);

// Root myDeserializedClass = JsonConvert.DeserializeObject<Root>(myJsonResponse);
public class AudioFile
{
    public int id { get; set; }
    public int chapter_id { get; set; }
    public double file_size { get; set; }
    public string format { get; set; }
    public string audio_url { get; set; }
    public int? duration { get; set; }
    public List<VerseTiming> verse_timings { get; set; }
}

public class Root
{
    public List<AudioFile> audio_files { get; set; }
}

public class VerseTiming
{
    public string verse_key { get; set; }
    public int timestamp_from { get; set; }
    public int timestamp_to { get; set; }
    public int? duration { get; set; }
    public List<List<double>> segments { get; set; }
}

#### Download from the server

In [21]:
var reciterId = 10;
var reciterName = "saud_ash-shuraym";

In [18]:
var urlBase = $"https://api.qurancdn.com/api/qdc/audio/reciters/{reciterId}/audio_files?segments=true&chapter=";
var downloadPath = @"D:\DatabaseBackups\QuranDB\recitaion-timings\downloaded-from-qurancdn";
var downloadFileName = downloadPath + $"\\{reciterName}_audioTimingFiles.json";

if (File.Exists(downloadFileName)){
    Console.WriteLine($"File Exists!");
    return;
}

var audioTimingFiles = new List<AudioFile>();

for(int i = 0; i < quranData.suras.Count; i++){
    var sura = quranData.suras[i];
    var serial = i + 1;

    using (var httpClient = new HttpClient()) {
        var apiUrl = urlBase + serial;
        var apiResponse = httpClient.GetAsync(apiUrl).Result;
        var apiContent = apiResponse.Content.ReadFromJsonAsync<Root>().Result;
        audioTimingFiles.AddRange(apiContent.audio_files);
    }
}

var resultJson = JsonSerializer.Serialize(audioTimingFiles, jsonSerializerOptions);

File.WriteAllText(downloadFileName, resultJson);

Console.WriteLine($"Download completed");

Download completed


#### Process from file

In [19]:
var timingJson = File.ReadAllText(@$"D:\DatabaseBackups\QuranDB\recitaion-timings\downloaded-from-qurancdn\{reciterName}_audioTimingFiles.json");
var audioTimingFiles = JsonSerializer.Deserialize<List<AudioFile>>(timingJson);
var downloadPath = @"D:\DatabaseBackups\QuranDB\recitaion-timings\downloaded-from-qurancdn";

In [20]:
public class RecitaionTiming {
    public int sura;
    public int ayat;
    public int time;
    public List<List<double>> wordTimings;
}
var recitaionTimings = new List<RecitaionTiming>();

foreach(var audioTimingFile in audioTimingFiles) {
    recitaionTimings.AddRange(audioTimingFile.verse_timings.Select(verse_timing=> {
        var verse_key = verse_timing.verse_key.Split(':');
        return new RecitaionTiming { 
            sura = int.Parse(verse_key[0]),
            ayat = int.Parse(verse_key[1]),
            time = verse_timing.timestamp_from,
            wordTimings = verse_timing.segments
        };
    }));
}

var arrayList = new List<object>();

foreach (var item in recitaionTimings){
    arrayList.Add(new object[]{ item.sura, item.ayat, item.time, item.wordTimings, "newLine" });
}

var resultJson = JsonSerializer.Serialize(arrayList)
                    .Replace(",\"newLine\"],", "]," + Environment.NewLine)
                    .Replace(",\"newLine\"","");

File.WriteAllText(downloadPath + $"\\{reciterName}_audioTimingFiles_transformed.json", resultJson);

audioTimingFiles.Count

114

### Downlaod from quran.gov.bd

In [ ]:
using System.IO;
using System.Text.Json;
using System.Net;

var srcPath = @"D:\Faruque\Projects\quranpwa\src\assets\";
var quranDataJson = File.ReadAllText(srcPath + "quran-data.json");

public class QuranData
{
    public List<List<object>> suras { get; set; }
    public List<List<int>> hizb_quarters { get; set; }
    public List<List<int>> manzils { get; set; }
    public List<List<int>> rukus { get; set; }
    public List<List<int>> pages { get; set; }
    public List<List<object>> sajdas { get; set; }
    public List<List<int>> juzs { get; set; }
}

var quranData = JsonSerializer.Deserialize<QuranData>(quranDataJson);

record DownloadableFile(string url, string filename);

var downlaodableFiles = new List<DownloadableFile>();

for(int i = 0; i < quranData.suras.Count; i++){
    var sura = quranData.suras[i];
    var serial = i + 1;
    //var start = (int)sura[0];
    var ayas = int.Parse(sura[1].ToString());

    for(int j = 1; j <= ayas; j++){
        downlaodableFiles.Add( new($"{serial}/{serial}-{j}.mp3", $"{serial.ToString().PadLeft(3, '0')}{j.ToString().PadLeft(3, '0')}.mp3"));
    }
}

var urlBase = "http://quran.gov.bd/quran/Sound/bangla";
var downloadPath = @"D:\Faruque\Projects\QuranAudio\Mahbub_Subhan_quran.gov.bd";

foreach (var downlaodableFile in downlaodableFiles){
    string url = $"{urlBase}/{downlaodableFile.url}";
    string filePath = $"{downloadPath}\\{downlaodableFile.filename}";

    if (File.Exists(filePath))
        continue;

    using (var client = new WebClient())
    {
        client.DownloadFile(url, filePath);
    }
    Console.WriteLine($"Downloaded file: {downlaodableFile}");
}

Console.WriteLine($"Download completed");
